In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
paths = []
dataset_gubuns = []
label_gubuns = []
# os.walk()를 이용하여 특정 디렉토리 밑에 있는 모든 하위 디렉토리를 모두 조사
# cat-and-dog 하위 디렉토리 밑에 jpg 확장자를 가진 파일이 모두 이미지 파일임
# cat-and-dong 밑으로 /train/, /test/ 하위 디렉토리 존재(학습, 테스트 용 이미지 파일들을 가짐)

for dirname, _, filenames in os.walk('/kaggle/input/cat-and-dog'):
    for filename in filenames:
    # 이미지 파일이 아닌 파일도 해당 디렉토리에 있음.
        if '.jpg' in filename:
            # 파일의 절대 경로를 file_path 변수에 할당.
            file_path = dirname+'/'+filename
            paths.append(file_path)
            # 파일의 절대 경로에 training_set, test_set가 포함되어 있으면 데이터 세트 구분을 'train'과 'test'로 분류.
            if 'training_set/' in file_path:
                dataset_gubuns.append('train')
            elif '/test_set/' in file_path:
                dataset_gubuns.append('test')
            else: dataset_gubuns.append('N/A')
                
            # 파일의 절대 경로에 dogs가 있을 경우 해당 파일은 dog 이미지 파일이고, cats일 경우는 cat 이미지 파일임.
            if 'dogs' in file_path:
                label_gubuns.append('DOG')
            elif 'cats' in file_path:
                label_gubuns.append('CAT')
            else: label_gubuns.append('N/A')
    
    

In [ ]:
paths[:10], dataset_gubuns[:10], label_gubuns[:10]

In [ ]:
pd.set_option('display.max_colwidth', 200)

data_df = pd.DataFrame({'path':paths, 'dataset':dataset_gubuns, 'label':label_gubuns})
print('data_df shape:', data_df.shape)
data_df.head(10)

In [ ]:
# 데이터 세트값 분포 및 Label 값 분포를 확인
print(data_df['dataset'].value_counts())
print(data_df['label'].value_counts())

In [ ]:
# DOG과 CAT의 이미지 파일 절대 경로를 cv2.imread()로 읽어서 image array로 로드하고 이미지 시각화
# 이미지별로 서로 다른 이미지 사이즈를 가지고 있음
import matplotlib.pyplot as plt
import cv2
%matplotlib inline

def show_grid_images(image_path_list, ncols=8, augmentor=None, title=None):
    figure, axs = plt.subplots(figsize=(22, 6), nrows=1, ncols=ncols)
    for i in range(ncols):
        image = cv2.cvtColor(cv2.imread(image_path_list[i]), cv2.COLOR_BGR2RGB)
        axs[i].imshow(image)
        axs[i].set_title(title)
        
dog_image_list = data_df[data_df['label']=='DOG']['path'].iloc[:6].tolist()
show_grid_images(dog_image_list, ncols=6, title='DOG')

cat_image_list = data_df[data_df['label'] == 'CAT']['path'].iloc[:6].tolist()
show_grid_images(cat_image_list, ncols=6, title='CAT')

In [ ]:
for image_path in dog_image_list:
    image = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
    print('image shape:', image.shape)
    
print('image height shape:', image[:, 0, 0].shape)
print('image width shape:', image[0, :, 0].shape)
print('image size:', image[:, :, 0].shape)

print("### image arrap:\n", image)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

### Preprocessing과 Data Loading을 동시에 적용. 단, 실제 Preprocessing과 Data Loading을 Model에서 fit_generator()을 호출하기 전까지는 동작하지 않음.
# augmentation은 horizontal_flip(좌우 반전)만 적용하고 0~255의 pixel값을 0 ~ 1 로 scale만 적용
train_gen = ImageDataGenerator(horizontal_flip=True, rescale=1/255.0)

# ImageDataGenerator 객체의 flow_from_directory() 메소드를 호출
# class_mode = 'categorical'로 Label 데이터를 원-핫 인코딩, 이미지 array는 224 x 224로 변경. Batch 크기는 64로 설정
train_flow_gen = train_gen.flow_from_directory(directory='/kaggle/input/cat-and-dog/training_set/training_set' # image file이 있는 디렉토리
                                               ,target_size=(224, 224) # 원본 이미지를 최종 resize 할 image size
                                               ,class_mode='categorical' # 문자열 label을 자동으로 one-hot-encoding 시켜줌
                                               ,batch_size=64
                                              )

In [ ]:
# next()를 flow_from_directory()로 반환된 Iterator 객체를 강제로 호출하여 어떤 값이 만들어지는지 확인
# 튜플 형태로 반환되고 튜플의 첫번째 값은 image array, 두 번째 값은 label array임
# image array는 0~1 사이 값인 float32로 변환됨. label array는 2개 column으로 원-핫 인코딩 됨
# next(iter(train_flow_gen))
image_array, labels_array = next(train_flow_gen)
print(image_array.shape, labels_array.shape)
print(image_array[:1], labels_array[:1])

In [ ]:
IMAGE_SIZE = 224
BATCH_SIZE = 64

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense , Conv2D , Dropout , Flatten , Activation, MaxPooling2D , GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam , RMSprop 
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau , EarlyStopping , ModelCheckpoint , LearningRateScheduler
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.applications import Xception

def create_model(model_name='vgg16', verbose=False):
    
    input_tensor = Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
    if model_name == 'vgg16':
        base_model = VGG16(input_tensor=input_tensor, include_top=False, weights='imagenet')
    elif model_name == 'resnet50':
        base_model = ResNet50V2(input_tensor=input_tensor, include_top=False, weights='imagenet')
    elif model_name == 'xception':
        base_model = Xception(input_tensor=input_tensor, include_top=False, weights='imagenet')
    
    bm_output = base_model.output

    x = GlobalAveragePooling2D()(bm_output)
    if model_name != 'vgg16':
        x = Dropout(rate=0.5)(x)
    x = Dense(50, activation='relu', name='fc1')(x)
    # 개와 고양이 2 종류이므로 Dense의 units는 2
    output = Dense(2, activation='softmax', name='output')(x)

    model = Model(inputs=input_tensor, outputs=output)
    
    if verbose:
        model.summary()
        
    return model


In [ ]:
model = create_model(model_name='xception', verbose=True)
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# train 데이터 건수는 generator의 samples 속성을 이용하여 가져올 수 있음
train_image_cnt = train_flow_gen.samples

# batch size는 64, 학습 데이터 image 개수는 8085
model.fit(train_flow_gen, epochs=15,steps_per_epoch=int(np.ceil(train_image_cnt/BATCH_SIZE)))



In [ ]:
# test data는 augmentation을 적용할 필요 없음.
test_gen = ImageDataGenerator(rescale=1/255.0)
test_flow_gen = test_gen.flow_from_directory(directory='/kaggle/input/cat-and-dog/test_set/test_set' # image file이 있는 디렉토리
                                               ,target_size=(IMAGE_SIZE, IMAGE_SIZE) # 원본 이미지를 최종 resize 할 image size
                                               ,class_mode='categorical' # 문자열 label을 자동으로 one-hot-encoding 시켜줌
                                               ,batch_size=BATCH_SIZE, shuffle=False
                                              )

In [ ]:
# 모델의 evaluate_generator()를 호출하여 Test 데이터의 성능 측정
model.evaluate_generator(test_flow_gen)

In [ ]:
pd.set_option('display.max_colwidth', 200)

data_df = pd.DataFrame({'path':paths, 'dataset':dataset_gubuns, 'label':label_gubuns})
print(data_df['dataset'].value_counts())
data_df.head(10)

In [ ]:
train_df = data_df[data_df['dataset']=='train']
test_df = data_df[data_df['dataset']=='test']
print('train_df shape:', train_df.shape, 'test_df shape:',test_df.shape)

In [ ]:
from sklearn.model_selection import train_test_split

# sckit learn의 train_test_split()을 이용하여 train용, validation용 DataFrame 생성
# stratify를 이용하여 label 값을 균등하게 분할
tr_df, val_df = train_test_split(train_df, test_size=0.15, stratify=train_df['label'], random_state=2021)
print('tr_df shape:', tr_df.shape, 'val_df shape', val_df.shape)
print('tr_df distribution:\n', tr_df['label'].value_counts())
print('val_df label distribution:\n', val_df['label'].value_counts())

In [ ]:
IMAGE_SIZE = 224
BATCH_SIZE = 64

In [ ]:
tr_generator = ImageDataGenerator(horizontal_flip=True, rescale=1/255.0)

tr_flow_gen = tr_generator.flow_from_dataframe(dataframe=tr_df
                                              ,x_col='path'
                                              ,y_col='label'
                                              ,target_size=(IMAGE_SIZE, IMAGE_SIZE)
                                              ,class_mode='binary'
                                              ,batch_size=BATCH_SIZE
                                              ,shuffle=True
                                              )

In [ ]:
images_array = next(tr_flow_gen)[0]
labels_array = next(tr_flow_gen)[1]
print('image array shape:', images_array.shape)
print('label array shape:', labels_array.shape)
print(images_array[0])
print(labels_array[0])

In [ ]:
# 검증용 ImageDataGenerator는 rescale만 적용.
val_generator = ImageDataGenerator(rescale=1/255.0)
val_flow_gen = val_generator.flow_from_dataframe(dataframe=val_df
                                                ,x_col='path'
                                                ,y_col='label'
                                                ,target_size=(IMAGE_SIZE, IMAGE_SIZE)
                                                ,class_mode='binary'
                                                ,batch_size=BATCH_SIZE
                                                ,shuffle=False
                                                )

In [ ]:
def create_model(model_name='vgg16', verbose=False):
    
    input_tensor = Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
    if model_name == 'vgg16':
        base_model = VGG16(input_tensor=input_tensor, include_top=False, weights='imagenet')
    elif model_name == 'resnet50':
        base_model = ResNet50V2(input_tensor=input_tensor, include_top=False, weights='imagenet')
    elif model_name == 'xception':
        base_model = Xception(input_tensor=input_tensor, include_top=False, weights='imagenet')
    
    bm_output = base_model.output

    x = GlobalAveragePooling2D()(bm_output)
    if model_name != 'vgg16':
        x = Dropout(rate=0.5)(x)
    x = Dense(50, activation='relu', name='fc1')(x)
    output = Dense(1, activation='sigmoid', name='output')(x)

    model = Model(inputs=input_tensor, outputs=output)
    
    if verbose:
        model.summary()
        
    return model

In [ ]:
model = create_model(model_name='xception')
model.compile(optimizer=Adam(0.001), loss='binary_crossentropy', metrics=['accuracy'])

rlr_cb = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, mode='min', verbose=1)
ely_cb = EarlyStopping(monitor='val_loss', patience=5, mode='min', verbose=1)

In [ ]:
N_EPOCHS = 15

model.fit(tr_flow_gen, epochs=N_EPOCHS,
         steps_per_epoch=int(np.ceil(tr_df.shape[0]/BATCH_SIZE)),
         validation_data=val_flow_gen, validation_steps=int(np.ceil(val_df.shape[0]/BATCH_SIZE)),
         callbacks=[rlr_cb, ely_cb])

In [ ]:
# test data는 augmentation 적용할 필요 없음
test_generator = ImageDataGenerator(rescale=1/255.0)
test_flow_gen = test_generator.flow_from_dataframe(dataframe=test_df
                                                  ,x_col='path'
                                                  ,y_col='label'
                                                  ,target_size=(IMAGE_SIZE, IMAGE_SIZE)
                                                  ,class_mode='binary'
                                                  ,batch_size=BATCH_SIZE
                                                  ,shuffle=False
                                                  )

# evaluate으로 성능 검증
model.evaluate(test_flow_gen)